# Topic Model Parts of Speech

This is a notebook for trying to use topic models for classifying sets of text that are more syntactically similar than topically similar. This notebook attempts to distinguish between discussion and conclusion section of scientific papers.

Below we are loading the dataset for use.

In [1]:
from __future__ import print_function
from time import time
import os

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.cross_validation import train_test_split

import numpy as np

import pickle

validDocsDict = dict()
fileList = os.listdir("PubMedPOS")
for f in fileList:
    validDocsDict.update(pickle.load(open("PubMedPOS/" + f, "rb")))

Here we are setting some vaiables to be used below and defining a function for printing the top words in a topic for the topic modeling.

In [2]:
n_samples = len(validDocsDict.keys())
n_features = 1000
n_topics = 2
n_top_words = 10


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Pre-process data

Here we are preprocessing data for use later. This code only grabs the discussion and conclusion sections of the data. We are also creating appropriate labels for the data and spliting the documents up to train and test sets.

In [3]:
print("Loading dataset...")
t0 = time()
documents = []

labels = []
concLengthTotal = 0
discLengthTotal = 0
concCount = 0
discCount = 0

for k in validDocsDict.keys():
    if k.startswith("conclusion"):
        labels.append("conclusion")
        documents.append(validDocsDict[k])
        concCount += 1
        concLengthTotal += len(validDocsDict[k].split(' '))
    elif k.startswith("discussion"):
        labels.append("discussion")
        documents.append(validDocsDict[k])
        discCount += 1
        discLengthTotal += len(validDocsDict[k].split(' '))

print(len(documents))
print(concLengthTotal * 1.0/ concCount)
print(discLengthTotal * 1.0/ discCount)

train, test, labelsTrain, labelsTest = train_test_split(documents, labels, test_size = 0.1)

Loading dataset...
25626
536.164364317
1219.64770155


Here we are splitting the data up some more to train different models. Discussion and conclusion sections are being put into their own training sets. A TFIDF vectorizer is trained with the whole dataset of conclusion AND discussion sections. The multiple different training sets are then transformed using this vectorizer to get vector encodings of the text normalized to sum to 1 which accounts for differing lengths of conclusion and discussion sections.

In [4]:
trainSetOne = []
trainSetTwo = []

for x in range(len(train)):
    if labelsTrain[x] == "conclusion":
        trainSetOne.append(train[x])
    else:
        trainSetTwo.append(train[x])

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
#tf_vectorizer = TfidfVectorizer(max_df=0.95, norm = 'l1', min_df=2, max_features=n_features)
tf_vectorizer = TfidfVectorizer(max_df=0.95, norm = 'l1', min_df=2, max_features=n_features, ngram_range = (1,4))
t0 = time()
tf = tf_vectorizer.fit_transform(train)

tfSetOne = tf_vectorizer.transform(trainSetOne)
tfSetTwo = tf_vectorizer.transform(trainSetTwo)
tfTest = tf_vectorizer.transform(test)
test = tfTest
train = tf
trainSetOne = tfSetOne
trainSetTwo = tfSetTwo

print("done in %0.3fs." % (time() - t0))

Extracting tf features for LDA...
done in 96.540s.


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


# LDA With Two Topics

Define an LDA topic model on the whole data set with two topics. This is trying to see if the topic model can define the difference between the two groups automatically and prints the top words per topic.

In [5]:
print("Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=100,
                                learning_method='online', learning_offset=50.,
                                random_state=0)

t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Fitting LDA models with tf features, n_samples=73447 and n_features=1000...
done in 424.516s.

Topics in LDA model:
Topic #0:
vbn rb dt jj vb jj nns vbd vbz to in dt nn dt jj nn
Topic #1:
cd nnp nnp nnp cd nn nn nn nn nn cd cd cd nnp nnp nnp jj nn nn in jj


Transform the unknown data through the topic model and calculate which topic it is more associated with according to the ratios. Calculate how many of each type (conclusion and discussion) go into each topic (1 or 2).

In [6]:
results = lda.transform(test)
totalConTop1 = 0
totalConTop2 = 0
totalDisTop1 = 0
totalDisTop2 = 0
for x in range(len(results)):
    val1 = results[x][0]
    val2 = results[x][1]
    total = val1 + val2
    print(str(labelsTest[x]) + " " + str(val1/total) + " " + str(val2/total))
    if val1 > val2:
        if labelsTest[x] == "conclusion":
            totalConTop1 += 1
        else:
            totalDisTop1 += 1
    else:
        if labelsTest[x] == "conclusion":
            totalConTop2 += 1
        else:
            totalDisTop2 += 1

conclusion 0.594287842989 0.405712157011
discussion 0.631908800015 0.368091199985
discussion 0.602628448732 0.397371551268
discussion 0.60443118111 0.39556881889
discussion 0.536503323287 0.463496676713
discussion 0.635616990248 0.364383009752
conclusion 0.614058997355 0.385941002645
discussion 0.651327020829 0.348672979171
conclusion 0.323512187783 0.676487812217
conclusion 0.498970973264 0.501029026736
discussion 0.60707038607 0.39292961393
discussion 0.597907373296 0.402092626704
conclusion 0.676527052373 0.323472947627
discussion 0.545827842704 0.454172157296
conclusion 0.655171779829 0.344828220171
conclusion 0.436451554946 0.563548445054
discussion 0.60528013888 0.39471986112
discussion 0.642019322873 0.357980677127
discussion 0.612397400593 0.387602599407
discussion 0.622804798314 0.377195201686
conclusion 0.540889405428 0.459110594572
discussion 0.600945121555 0.399054878445
conclusion 0.380497602438 0.619502397562
conclusion 0.56909493189 0.43090506811
conclusion 0.35040975417

Print out the results from the topic transforms.

In [7]:
print("Total Conclusion Topic One: " + str(totalConTop1))
print("Total Conclusion Topic Two: " + str(totalConTop2))
print("Total Discussion Topic One: " + str(totalDisTop1))
print("Total Discussion Topic Two: " + str(totalDisTop2))

Total Conclusion Topic One: 641
Total Conclusion Topic Two: 630
Total Discussion Topic One: 1153
Total Discussion Topic Two: 139


Get the parameters for the LDA.

In [8]:
lda.get_params()

{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 50.0,
 'max_doc_update_iter': 100,
 'max_iter': 100,
 'mean_change_tol': 0.001,
 'n_jobs': 1,
 'n_topics': 2,
 'perp_tol': 0.1,
 'random_state': 0,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}

# Basic Classifiers

Train three basic classifiers to solve the problem. Try Gaussian, Bernoulli and K Nearest Neighbors classifiers and calculate how accurate they are.

In [9]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()

classifier.fit(train.toarray(), labelsTrain)

classResults = classifier.predict(test.toarray())

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.888021849395


In [10]:
from sklearn.naive_bayes import BernoulliNB

classifier = BernoulliNB()

classifier.fit(train.toarray(), labelsTrain)

classResults = classifier.predict(test.toarray())

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.86149044089


In [11]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()

classifier.fit(train, labelsTrain)

classResults = classifier.predict(test)
numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.728833398361


In [12]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()

classifier.fit(train.toarray(), labelsTrain)

classResults = classifier.predict(test.toarray())
numRight = 0
numWrongDisc = 0
numWrongConc = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1
    else:
        if classResults[item] == "discussion":
            numWrongDisc += 1
        else:
            numWrongConc += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))
print("Incorrectly classified as discussion: " + str(numWrongDisc))
print("Incorrectly classified as conclusion: " + str(numWrongConc))
print(len(classResults))

0.918454935622
Incorrectly classified as discussion: 104
Incorrectly classified as conclusion: 105
2563


# Two Topic Models

Define two topic models with 20 topics each, one on discussion sections and one on conclusion sections. Then transform both the train and test sets using both topic models to get 40 features for each sample based on the probability distribution for each topic in each LDA.

In [13]:
ldaSet1 = LatentDirichletAllocation(n_topics=20, max_iter=100,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
ldaSet2 = LatentDirichletAllocation(n_topics=20, max_iter=100,
                                learning_method='online', learning_offset=50.,
                                random_state=0)

In [14]:
ldaSet1.fit(trainSetOne)
print_top_words(ldaSet1, tf_feature_names, n_top_words)

Topic #0:
jj nn vbp nn jj nns vbp dt jj nnp nn nnp jj nn nn jj nnp jj nn nn nns jj nn nn jj nn nns jj nn nn nn vbp
Topic #1:
jj nn vbp nn jj nns vbp dt jj nnp nn nnp jj nn nn jj nnp jj nn nn nns jj nn nn jj nn nns jj nn nn nn vbp
Topic #2:
cd cd cd cd cd cd cd cd cd cd cd nn cd cd nn nn cd cd nn cd cd cd cd nn cd nn cd cd
Topic #3:
jj nn vbp nn jj nns vbp dt jj nnp nn nnp jj nn nn jj nnp jj nn nn nns jj nn nn jj nn nns jj nn nn nn vbp
Topic #4:
jj nn vbp nn jj nns vbp dt jj nnp nn nnp jj nn nn jj nnp jj nn nn nns jj nn nn jj nn nns jj nn nn nn vbp
Topic #5:
jj nn vbp nn jj nns vbp dt jj nnp nn nnp jj nn nn jj nnp jj nn nn nns jj nn nn jj nn nns jj nn nn nn vbp
Topic #6:
jj nn vbp nn jj nns vbp dt jj nnp nn nnp jj nn nn jj nnp jj nn nn nns jj nn nn jj nn nns jj nn nn nn vbp
Topic #7:
jj nn vbp nn jj nns vbp dt jj nnp nn nnp jj nn nn jj nnp jj nn nn nns jj nn nn jj nn nns jj nn nn nn vbp
Topic #8:
jj nn vbp nn jj nns vbp dt jj nnp nn nnp jj nn nn jj nnp jj nn nn nns jj nn nn jj nn nns jj

In [15]:
ldaSet2.fit(trainSetTwo)
print_top_words(ldaSet2, tf_feature_names, n_top_words)

Topic #0:
nn nnp nn nnp nnp jj nnp nn nn nn nnp jj nn nn nn nn nnp nn nn nn jj nn nn jj jj nn nnp nnp nnp fw fw fw fw nnp nnp nnp in
Topic #1:
nn nn nn nnp nnp jj nn nn nn jj jj nn nn nn jj nn nnp nn nnp nn nn nn nnp nnp jj nnp cd cd cd nnp in nnp nnp jj nnp nnp
Topic #2:
fw fw fw fw fw fw fw fw fw fw nn fw nn fw fw fw fw nn fw nn nn nn nn nnp
Topic #3:
nn nnp nn nn nn jj jj nnp jj nn nnp nnp jj nn nn nn nnp nn nn nn jj nnp nn nn nn fw fw fw fw nn nnp nnp nnp nnp nn nn
Topic #4:
nn nnp nn nnp nnp jj nnp nn nn nn nnp jj nn nn nn nn nnp nn nn nn jj nn nn jj jj nn nnp nnp nnp fw fw fw fw nnp nnp nnp in
Topic #5:
nn nn jj jj nnp nnp jj nn nn nn jj nn nnp nn nn nn nn nnp nnp jj nn nnp nn nn nn nn nn nnp nnp nnp nnp cd nn nnp nnp cd cd
Topic #6:
nn nn jj jj nnp nn nn nn nn nnp nn nnp nnp jj nnp jj nn nn nn nn nnp nn nn nn jj nnp cd cd cd nnp cd nn nn nnp nn nn
Topic #7:
nn nnp nn nn nn jj jj nnp jj nn nnp nnp jj nn nn nn nnp nn nn nn jj nnp nn nn nn fw fw fw fw nn nnp nnp nnp nnp nn nn
Topic

In [16]:
results1 = ldaSet1.transform(train)
results2 = ldaSet2.transform(train)

resultsTest1 = ldaSet1.transform(test)
resultsTest2 = ldaSet2.transform(test)

In [17]:
results = np.hstack((results1, results2))
resultsTest = np.hstack((resultsTest1, resultsTest2))

Define two classifiers using the transformed train and test sets from the topic models. Print out the accuracy of each one.

In [18]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()

classifier.fit(results, labelsTrain)

classResults = classifier.predict(resultsTest)

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.503316426063


In [19]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()

classifier.fit(results, labelsTrain)

classResults = classifier.predict(resultsTest)

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.747561451424


Normalize the results of each sample of 40 features so they sum to 1. Then train two more classifiers using the data and print out the accuracy of each.

In [20]:
for x in range(len(results)):
    total = 0
    for y in range(len(results[x])):
        total += results[x][y]
    for y in range(len(results[x])):
        results[x][y] = results[x][y]/total
        
for x in range(len(resultsTest)):
    total = 0
    for y in range(len(resultsTest[x])):
        total += resultsTest[x][y]
    for y in range(len(resultsTest[x])):
        resultsTest[x][y] = resultsTest[x][y]/total

In [21]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()

classifier.fit(results, labelsTrain)

classResults = classifier.predict(resultsTest)

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.503316426063


In [22]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()

classifier.fit(results, labelsTrain)

classResults = classifier.predict(resultsTest)

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.749122122513
